# 라이브러리 로드

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler # learning rate를 학습 중간중간에 바꿀 수 있게 해주는 함수
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy

import cv2
import torchvision.transforms.functional as F

# 모델 형태 선언

In [2]:
model_ft = models.resnet18(weights=True)
num_features = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_features, 2) 
# 위의 코드를 보면 마지막 레이어의 상태가 (fc): Linear(in_features=512, out_features=1000, bias=True) 였는데, 
# 이걸 (fc): Linear(in_features=512, out_features=2, bias=True) 로 바꿔준 것이다.

/home/kengwon/ken/EOF_SeparateTrashCollection/ArtificialNeuralNetwork/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# 모델 파라미터 load

In [4]:
PATH = './eof_bottle_classification_model.pth'
model_ft.load_state_dict(torch.load(PATH))

<All keys matched successfully>

# 모델이 원하는 인풋 형상으로 변환시켜주기 위한 transforms

In [5]:
# 내가 학습시킨 모델이 원하는 인풋 현상
test_data_transforms = transforms.Compose([transforms.ToPILImage(), ## opencv로 이미지 읽어서 돌릴꺼면 pil 이미지로 한번 변환해줘야함
                                         transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(), 
                                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                         ])

# inputs 이미지 로드 (openCV 이미지 활용)

In [6]:
test_img = cv2.imread("test_input4.jpg")
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_tensor = test_data_transforms(test_img)

In [7]:
model_ft.eval()
with torch.no_grad():
    # test_tensor = test_tensor.to(device)
    outputs = model_ft(test_tensor.unsqueeze(0)) # 모델은 현재 batch 디멘션이 포함된 4D 형태의 인풋을 기대하고 있음
    _, predicts = torch.max(outputs, 1)

In [8]:
_

tensor([5.7508])

In [10]:
class_names = ['nl_bottle', 'yl_bottle']
class_names[predicts[0]]

'yl_bottle'